In [1]:
# 랜덤포레스트라이브러리
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#geopandas
import geopandas as gpd
import pandas as pd

# 공간연산
from shapely.geometry import Point
